In [12]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import os
import random
import numpy as np

# CLEAN

In [13]:
df = pd.read_csv('ebay.csv')
df.head()

,ePID,Make,Model,Model_Submodel,Submodel,Year,Vehicle Type,Motorcycle Type
0,110729864,Husaberg,FC550,FC550,--,2004,Motorcycle,Off-road Motorcycle
1,110729874,Spencer Bowman Customs,Rock Bottom,Rock Bottom Pro Street ST,Pro Street ST,2009,Motorcycle,Street Motorcycle
2,110729884,Tank Sports,Racer,Racer 150,150,2009,Scooter,Scooter
3,110729904,Derbi,Senda,Senda SM,SM,1999,Motorcycle,Dual Sport Motorcycle
4,110729914,BMC,Notorious,Notorious 918,918,2003,Motorcycle,Street Motorcycle


In [14]:
def reverse_sentence(sentence):
    return " ".join(sentence.split()[::-1])

df['Model_Submodel_inv'] = df['Model_Submodel'].apply(reverse_sentence)
df['Model_inv'] = df['Model'].apply(reverse_sentence)
df.head()

,ePID,Make,Model,Model_Submodel,Submodel,Year,Vehicle Type,Motorcycle Type,Model_Submodel_inv,Model_inv
0,110729864,Husaberg,FC550,FC550,--,2004,Motorcycle,Off-road Motorcycle,FC550,FC550
1,110729874,Spencer Bowman Customs,Rock Bottom,Rock Bottom Pro Street ST,Pro Street ST,2009,Motorcycle,Street Motorcycle,ST Street Pro Bottom Rock,Bottom Rock
2,110729884,Tank Sports,Racer,Racer 150,150,2009,Scooter,Scooter,150 Racer,Racer
3,110729904,Derbi,Senda,Senda SM,SM,1999,Motorcycle,Dual Sport Motorcycle,SM Senda,Senda
4,110729914,BMC,Notorious,Notorious 918,918,2003,Motorcycle,Street Motorcycle,918 Notorious,Notorious


In [15]:
new_columns_name = [column.lower().replace(':','') for column in df.columns]
df.columns=new_columns_name
df.dropna(subset=['model'], inplace=True)
df.rename(columns = {'make':'brand', 'motorcycle type': 'category', 'vehicle type':'type'}, inplace = True)

# brand and model cleaning
df.model_submodel_inv = df.model_submodel_inv.str.lower()
df.model_submodel = df.model_submodel.str.lower()
df.model = df.model.str.lower()
df.model_inv = df.model_inv.str.lower()
df.brand = df.brand.str.lower()
df.model_submodel_inv = df.model_submodel_inv.str.strip()
df.model_submodel = df.model_submodel.str.strip()
df.model = df.model.str.strip()
df.model_inv = df.model_inv.str.strip()
df.brand = df.brand.str.strip()
df.model_submodel_inv = df.model_submodel_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_submodel = df.model_submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.model_inv = df.model_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
df.brand = df.brand.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")

#clean category
df.category = df.category.str.lower().str.strip().str.replace('motorcycle', '')

#clean type
df.type = df.type.str.lower().str.strip()

# extra cleaning
df.dropna(subset=['year', 'brand', 'model'],inplace=True)
df.year = df.year.astype(int)
df.columns = [x+"_db" for x in df.columns]

df.drop(columns=['epid_db','submodel_db'], inplace=True)

df.to_csv('ebay.csv', index=False)

<ipython-input-15-e44f507c577e>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_submodel_inv = df.model_submodel_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-15-e44f507c577e>:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_submodel = df.model_submodel.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-15-e44f507c577e>:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model = df.model.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-15-e44f507c577e>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df.model_inv = df.model_inv.str.replace(r"\(.*\)","").str.replace('[^\w\s]','').str.replace(" ","")
<ipython-input-15-e44f507c577e>:21: FutureWarning: T

# IMPORT AND VISUALIZE

In [7]:
data = pd.read_csv('ebay.csv')
data.head()

,brand_db,model_db,model_submodel_db,year_db,type_db,category_db,model_submodel_inv_db
0,husaberg,fc550,fc550,2004,motorcycle,off-road,fc550
1,spencerbowmancustoms,rockbottom,rockbottomprostreetst,2009,motorcycle,street,ststreetprobottomrock
2,tanksports,racer,racer150,2009,scooter,scooter,150racer
3,derbi,senda,sendasm,1999,motorcycle,dual sport,smsenda
4,bmc,notorious,notorious918,2003,motorcycle,street,918notorious


In [ ]:
data[data.raw_model.str.contains('trom')]